In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('../data/train.csv')

df = df[df.price > 0]
df = df[df.lat<=1.47]
df = df[df.size_sqft>0]

df['price_per_sq_ft'] = df['price']/df['size_sqft']

df = df[['lat','lng', 'price_per_sq_ft']]
df['price_per_sq_ft'] = df['price_per_sq_ft'].astype(np.int32)

In [3]:
from math import cos, radians

def get_lat_offset(distance_in_m):
    offset = distance_in_m/111111
    return offset

def get_lng_offset(distance_in_m, lat):
    offset = distance_in_m/(111111*cos(radians(lat)))
    return offset

In [4]:
def get_bounding_square(lat, lng, side):
    lat_offset = get_lat_offset(side/2)
    lng_offset = get_lng_offset(side/2, lat)
    
    top_lat = lat + lat_offset
    bottom_lat = lat - lat_offset
    
    right_lng = lng + lng_offset
    left_lng = lng - lng_offset
    
    return top_lat, bottom_lat, right_lng, left_lng

In [5]:
bbox_side = 1000

def get_average_price_psf_within_bbox(lat, lng):
    bbox = get_bounding_square(lat, lng, bbox_side)
    filtered_df = df[(df.lat < bbox[0]) & (df.lat > bbox[1]) & (df.lng < bbox[2]) & (df.lng > bbox[3])]
    return filtered_df['price_per_sq_ft'].mean().astype(np.int32)

In [17]:
#steps from here on are what we need to do in order to calculate the psf price feature for a dataset
df_aug = pd.read_csv('../data/train.csv')

df_aug = df_aug[df_aug.price > 0]
df_aug = df_aug[df_aug.lat<=1.47]
df_aug = df_aug[df_aug.size_sqft>0]